IMPORTAÇÃO DE MODULOS

In [2]:
from path_arquivos import *
import pandas as pd

caminho = r'c:\WS_OLIVEIRA\OUTPUT\analitico_baixa.xlsx'


LEITURA DOS DATAFRAMES

In [3]:
df = pd.read_excel(caminho, sheet_name= 'FIM_MESA')

CORPO DO SCRIPT

In [ ]:
lista_fim = [47335286, 47336582, 47336594, 47341637, 47336912,47336817, 47336819,47341629, 47337359, 47344447]
lista_pd = []
fim = df.loc[df['NUMOS'].isin(lista)]

VISUALIZAÇÃO

In [ ]:
fim.to_excel(output.jupyter, index=False, sheet_name= "FIM_MESA")